# Machine Learning for Genomics Classification of SARS-CoV-2 variants

**Participants**

1. Mike Mwanga
2. Evans Mudibo
3. Hesbon Omwondho
4. Awe Oleitan


In [15]:
#Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from numpy import array

from sklearn.preprocessing import LabelEncoder #applied in 1D type of dtaset
from sklearn.preprocessing import OneHotEncoder #applied on 2D datasets
from sklearn.preprocessing import LabelBinarizer #applied in 1D type of dtaset

In [16]:
seq_data_aligned = pd.read_table('../data/seq_aligned.txt')
seq_data_not_aligned = pd.read_table('../data/seq_not_aligned.txt')

## Approach 1

### Convert the sequences into kmers. Then binarize the kmers. Binary data is then fed into the CNN model

In [17]:
data = [seq_data_aligned, seq_data_not_aligned]
sequence_data = [seq_data_aligned, seq_data_not_aligned]


In [18]:
#Convert variant ID to numeric and remove sequence ID

variant = {"gamma" : 1, "delta" : 2, "beta":3, "Alpha":4, "omicron":5 }

for seq_data in sequence_data:
    seq_data["Variant_Id"] = seq_data["Variant_Id"].map(variant)

### Convert the sequence into k-mers.

In [19]:
#function to convert sequences into kmers

def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]


for seq_data in sequence_data:
    seq_data['Seq'] = seq_data.apply(lambda x: getKmers(x['Sequence']), axis=1)


seq_data_not_aligned.head()

,Seq_Id,Sequence,Variant_Id,Seq
0,hCoV19/gamma/Italy/CAMUniSa10/2021|EPI_ISL_101...,attaaaggtttatacccttcccaggtaacaaaccaaccaactttcg...,1,"[attaaa, ttaaag, taaagg, aaaggt, aaggtt, aggtt..."
1,hCoV19/gamma/Italy/CAMUniSa23/2021|EPI_ISL_101...,attaaaggtttatacccttcccaggtaacaaaccaaccaactttcg...,1,"[attaaa, ttaaag, taaagg, aaaggt, aaggtt, aggtt..."
2,hCoV19/gamma/Italy/CAMUniSa111/2021|EPI_ISL_10...,attaaaggtttatacccttcccaggtaacaaaccaaccaactttcg...,1,"[attaaa, ttaaag, taaagg, aaaggt, aaggtt, aggtt..."
3,hCoV19/gamma/South Korea/NMCnCoV09/2021|EPI_IS...,agatctgttctctaaacgaactttaaaatctgtgtggctgtcactc...,1,"[agatct, gatctg, atctgt, tctgtt, ctgttc, tgttc..."
4,hCoV19/gamma/Brazil/PRBT74803FI/2021|EPI_ISL_9...,tagatctgttctctaaacgaactttaaaatctgtgtggctgtcact...,1,"[tagatc, agatct, gatctg, atctgt, tctgtt, ctgtt..."


##### Converting sequences into array of numericals using label encoder. It assigns numericals based on alphaetical order. i.e "AAAAA" = 0,"AAAAC" = 2. Each uniqure k-mer will be assigned a numerical value.

In [20]:
lb = LabelBinarizer()
for seq_data in sequence_data:
    encoded = []
    for seq in seq_data["Seq"]:
        value = array(seq)
        encoded.append(lb.fit_transform(value))
    seq_data["Seq_Encoded"] = encoded

In [21]:
sequence_data[1]

,Seq_Id,Sequence,Variant_Id,Seq,Seq_Encoded
0,hCoV19/gamma/Italy/CAMUniSa10/2021|EPI_ISL_101...,attaaaggtttatacccttcccaggtaacaaaccaaccaactttcg...,1,"[attaaa, ttaaag, taaagg, aaaggt, aaggtt, aggtt...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,hCoV19/gamma/Italy/CAMUniSa23/2021|EPI_ISL_101...,attaaaggtttatacccttcccaggtaacaaaccaaccaactttcg...,1,"[attaaa, ttaaag, taaagg, aaaggt, aaggtt, aggtt...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,hCoV19/gamma/Italy/CAMUniSa111/2021|EPI_ISL_10...,attaaaggtttatacccttcccaggtaacaaaccaaccaactttcg...,1,"[attaaa, ttaaag, taaagg, aaaggt, aaggtt, aggtt...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,hCoV19/gamma/South Korea/NMCnCoV09/2021|EPI_IS...,agatctgttctctaaacgaactttaaaatctgtgtggctgtcactc...,1,"[agatct, gatctg, atctgt, tctgtt, ctgttc, tgttc...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,hCoV19/gamma/Brazil/PRBT74803FI/2021|EPI_ISL_9...,tagatctgttctctaaacgaactttaaaatctgtgtggctgtcact...,1,"[tagatc, agatct, gatctg, atctgt, tctgtt, ctgtt...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...
81,hCoV19/omicron//Scotland/QEUH373F79A/2022|EPI_...,ttgtagatctgttctctaaacgaactttaaaatctgtgtggctgtc...,5,"[ttgtag, tgtaga, gtagat, tagatc, agatct, gatct...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
82,hCoV19/omicron//England/MILK3740F70/2022|EPI_I...,ttgtagatctgttctctaaacgaactttaaaatctgtgtggctgtc...,5,"[ttgtag, tgtaga, gtagat, tagatc, agatct, gatct...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
83,hCoV19/omicron//England/MILK373E151/2022|EPI_I...,ttgtagatctgttctctaaacgaactttaaaatctgtgtggctgtc...,5,"[ttgtag, tgtaga, gtagat, tagatc, agatct, gatct...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
84,hCoV19/omicron//Japan/TMDU_No229/2022|EPI_ISL_...,ttttacaggttcgcgacgtgctcgtacgtggctttggagactccgt...,5,"[ttttac, tttaca, ttacag, tacagg, acaggt, caggt...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [22]:
seq_data_aligned = seq_data_aligned.drop(["Seq_Id", "Sequence","Seq"], axis=1)
seq_data_not_aligned = seq_data_not_aligned.drop(["Seq_Id", "Sequence","Seq"], axis=1)


In [23]:
seq_data_aligned.head()

,Variant_Id,Seq_Encoded
0,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,1,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,1,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


## **Approach 2

Given than in real-world scenarios, we will come across sequences with ambigous bases, and undetermined nucleotids, converting these into zeros by mapping will be a reasonable approach. 

### Convert each nucleotide to numeric values so that we maintain order of sequence in the dataset**

Whether this will result to any different result in the model is something that would require further evalaution. Lets go

Here we assign each nucleotide a numeric value and any ambigous bases, Ns or '-' will assigned to zeros.

A-1
C-2
G-3
T-4
N - 0
Ambigous bases - 0
Dashes(gaps)-0


In [24]:
mapping_dic = {'A':1, "C":2, "G" : 3,"T" : 4, "-" : 0, "N":0, "R" :0, "Y" : 0, "K":0, "M":0, "S":0, "W":0, "B":0}

def convert_to_numeric(seq):
    """
    Function to convert nucleotide sequence into numeric values.

    For aligned sequences, will also be considered non-informative.
    """
    seq_numeric = []
    for base in seq:
        seq_numeric.append(mapping_dic[base.upper()])
    return ''.join(str(i) for i in seq_numeric)


Now convert the sequences in our dataset into numeric, then convert to binary form.

In [25]:
seq_data_aligned = pd.read_table('../data/seq_aligned.txt')
seq_data_not_aligned = pd.read_table('../data/seq_not_aligned.txt')

In [26]:
data = [seq_data_aligned,seq_data_not_aligned]

In [27]:
for seq_data in data:
    seq_data["Variant_Id"] = seq_data["Variant_Id"].map(variant)

In [28]:
#Apply in our dataset
for seq_data in data:
    seq_data['Seq_Numeric'] = seq_data.apply(lambda x: convert_to_numeric(x['Sequence']), axis=1)

    seq_data['Seq_kmers'] = seq_data.apply(lambda x: getKmers(x['Seq_Numeric']), axis=1)

    #numeric_data = numeric_data.drop(['Seq_Numeric','Seq_Binary'], axis=1)


In [29]:
lb = LabelBinarizer()
for seq_data in data:
    encoded = []
    for seq in seq_data["Seq_kmers"]:
        value = array(seq)
        encoded.append(lb.fit_transform(value))
    seq_data["Seq_Encoded"] = encoded

In [30]:
seq_data_aligned = seq_data_aligned.drop(["Seq_Id", "Sequence", "Seq_Numeric","Seq_kmers"], axis=1)
seq_data_not_aligned = seq_data_not_aligned.drop(["Seq_Id", "Sequence", "Seq_Numeric", 'Seq_kmers'], axis=1)

seq_data_aligned.head()

,Variant_Id,Seq_Encoded
0,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,1,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,1,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,1,"[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [33]:
#write to table

seq_data_aligned.to_csv("../data/sequene_encoded_aligned.txt", header=True, sep="\t", index=False)
seq_data_not_aligned.to_csv("../data/sequene_encoded_not_aligned.txt", header=True, sep="\t", index=False)
